# MILVUS Demo - Filter Templating

# Filter Templating during Search with Milvus in watsonx.data

## Disclaimers
- Use only Projects and Spaces that are available in watsonx context.

## Overview
### Audience
This notebook demonstrates how to implement Filter Templating.
To reduces the time spent parsing complex expressions, especially those with large arrays or non-ASCII characters Filter Templating could be used.It improves query performance and simplifies handling dynamic values across search, query, and delete operations.
The scenario presented in this notebook shows how to use Filter Templating during search.

Some familiarity with Python programming, search algorithms, and basic machine learning concepts is recommended. The code runs with Python 3.10 or later.
### Learning goal
This notebook demonstrates Milvus Filter Templating support in watsonx.data, introducing commands for:
- Connecting to Milvus
- Creating collections
- Creating indexes
- Generate Embeddings
- Ingesting data
- Data retrieval


### About Milvus 

Milvus is an open-source vector database designed specifically for scalable similarity search and AI applications. It's a powerful platform that enables efficient storage, indexing, and retrieval of vector embeddings, which are crucial in modern machine learning and artificial intelligence tasks.[ To know more, visit Milvus Documentation](https://www.ibm.com/docs/en/watsonx/watsonxdata/2.1.x?topic=components-milvus)

### Milvus: Three Fundamental Steps

#### 1. Data Preparation
Collect and convert your data into high-dimensional vector embeddings. These vectors are typically generated using machine learning models like neural networks, which transform text, images, audio, or other data types into dense numerical representations that capture semantic meaning and relationships.

#### 2. Vector Insertion
Load the dense vector embeddings and sparse vector embeddings into Milvus collections or partitions within a database. Milvus creates indexes to optimize subsequent search operations, supporting various indexing algorithms like IVF-FLAT, HNSW, etc., based on the definition.

#### 3. Similarity Search
Perform vector similarity searches by providing a query vector and a reranking weight. Milvus will rapidly return the most similar vectors from the collection or partitions based on the defined metrics like cosine similarity, Euclidean distance, or inner product and the reranking weight.

### What is Filter Templating?
Filter templating in Milvus is a mechanism that allows you to define filter expressions with placeholders. These placeholders can be dynamically replaced with actual values during query execution, enabling flexible and efficient query building. It improves query performance and simplifies handling dynamic values across search, query, and delete operations.

### Key Workflow

1. **Definition** (once)
2. **Ingestion** (once)
3. **Retrieve relevant passage(s)** (for every user query)

## Contents

- Environment Setup
- Install packages
- Document data loading
- Create connection
- Ingest data
- Retrieve relevant data

## Environment Setup
Before using the sample code in this notebook, complete the following setup tasks:

- Create a Watsonx.data instance (a free plan is offered)
  - Information about creating a watsonx.data instance can be found [here](https://www.ibm.com/docs/en/watsonx/watsonxdata/2.0.x)


## Install required packages

In [46]:
%%capture
!pip install numpy

In [47]:
%%capture
!pip install torch

In [48]:
%%capture
!pip install sentence-transformers

### Install Pymilvus SDK

In [49]:
# !pip install pymilvus
# Restart Kernal
%pip show pymilvus

Name: pymilvus
Version: 2.5.3
Summary: Python Sdk for Milvus
Home-page: 
Author: 
Author-email: Milvus Team <milvus-team@zilliz.com>
License: 
Location: /opt/homebrew/lib/python3.11/site-packages
Requires: grpcio, milvus-lite, pandas, protobuf, python-dotenv, setuptools, ujson
Required-by: 
Note: you may need to restart the kernel to use updated packages.


### Post pymilvus installations

In [50]:
import psutil
import sys
import numpy as np
import time
import pandas as pd
from pymilvus import MilvusClient, DataType, CollectionSchema, FieldSchema,utility,connections,Collection

In [51]:
%%capture
%pip install "pymilvus[model]"

In [52]:
%%capture
%pip install tensorflow

In [53]:
%%capture
%pip install --upgrade transformers

## Preparing data

In [54]:
%%capture
%pip install bs4

In [55]:
from bs4 import BeautifulSoup

# Load the CSV file
file_path = '/Users/home/milvusD/milvus/features_2.4.0/data/cleanFashion Dataset.csv'  # Replace with your CSV file path
columns_to_extract = ['p_id', 'name', 'price', 'colour','avg_rating','description']  # Replace with your desired column names

# Read the CSV file and extract selected columns
df = pd.read_csv(file_path, usecols=columns_to_extract, keep_default_na=False).replace('', 0).head(50)


In [56]:

def clean_html(html):
    if not isinstance(html, str):  # Handle non-string values
        html = str(html)
    return BeautifulSoup(html, "html.parser").get_text()

# Apply the function to the 'description' column
df['cleaned_description'] = df['description'].apply(clean_html)

# Drop the 'description' column
df = df.drop(columns=['description'])

# Save the extracted data to a new CSV (optional)
output_file_path = 'extracted_columns.csv'
df.to_csv(output_file_path, index=False)  


In [57]:
df.head(5)

,p_id,name,price,colour,avg_rating,cleaned_description
0,100001,Black Ethnic Motifs Printed Kurta with Palazzo...,5099,Black,4.418398939,Black printed Kurta with Palazzos with dupatta...
1,100002,Orange Solid Kurta with Palazzos & Floral Prin...,5899,Orange,4.11933395,Orange solid Kurta with Palazzos with dupattaK...
2,100003,Navy Blue Ethnic Motifs Embroidered Thread Wor...,4899,Navy Blue,4.16152968,Navy blue embroidered Kurta with Trousers with...
3,100004,Red Floral Printed Kurta With Trouser & Dupatta,3699,Red,4.088986142,Red printed kurta with trouser and dupattaKurt...
4,100005,Black & Green Printed Straight Kurta,1350,Black,3.978377362,"Black and green printed straight kurta, has a ..."


#### Setting Up BM25 and SentenceTransformer for Text Analysis

In [58]:
from pymilvus.model.sparse.bm25.tokenizers import build_default_analyzer
from pymilvus.model.sparse import BM25EmbeddingFunction

In [59]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

In [60]:

analyzer = build_default_analyzer(language="en")

bm25_ef = BM25EmbeddingFunction(analyzer)

### Connect to Milvus

In [61]:
from pymilvus import MilvusClient
# Replace Placeholder Values <> with respective provisioned Milvus Values .

uri = "https://<host>:<port>"  # Construct URI from host and port
user = "<>"
password = "<>"
# Create an instance of the MilvusClient class with the new configuration
"""
#On Prem
milvus_client = MilvusClient(uri=uri,
                            user=user,
                            password=password,
                            secure=True,
                            server_pem_path='<>',
                            server_name='<>',)

# SaaS
milvus_client = MilvusClient(uri=uri, 
                             user=user, 
                             password=password,
                             secure=True,
                             server_name='<>',)
"""

In [62]:
COLLECTION_NAME = "Milvus_test_scalar_filter"
DIMENSION = 384
BATCH_SIZE = 2
TOPK = 1
fmt = "=== {:30} ==="
search_latency_fmt = "search latency = {:.4f}s"

In [63]:
if milvus_client.has_collection(collection_name=COLLECTION_NAME):
    milvus_client.drop_collection(collection_name=COLLECTION_NAME)

In [64]:
milvus_client.has_collection(collection_name=COLLECTION_NAME)

False

## Create Milvus schema 
[more about schema](https://www.ibm.com/docs/en/watsonx/watsonxdata/2.1.x?topic=milvus-connecting-service#taskconctmilvus__postreq__1)

In [65]:
# Create schema
schema = milvus_client.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)
# [pid,name,cleaned_description,price,avg_rating,colour,name_embedding,description_embedding]
# Add fields to schema
schema.add_field(field_name="p_id", datatype=DataType.INT64, is_primary=True),
schema.add_field(field_name="name", datatype=DataType.VARCHAR, max_length=65535),
schema.add_field(field_name="cleaned_description", datatype=DataType.VARCHAR, max_length=65535),
schema.add_field(field_name="price",datatype=DataType.FLOAT)
schema.add_field(field_name="avg_rating",datatype=DataType.FLOAT,nullable=True )
schema.add_field(field_name="colour",datatype=DataType.VARCHAR, max_length=25)
schema.add_field(field_name="name_embedding", datatype=DataType.SPARSE_FLOAT_VECTOR),
schema.add_field(field_name="description_embedding", datatype=DataType.FLOAT_VECTOR, dim=DIMENSION)

{'auto_id': False, 'description': '', 'fields': [{'name': 'p_id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'name', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'cleaned_description', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'price', 'description': '', 'type': <DataType.FLOAT: 10>}, {'name': 'avg_rating', 'description': '', 'type': <DataType.FLOAT: 10>, 'nullable': True}, {'name': 'colour', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 25}}, {'name': 'name_embedding', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>}, {'name': 'description_embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'enable_dynamic_field': True}

## Create Index 
[more on indexes](https://www.ibm.com/docs/en/watsonx/watsonxdata/2.1.x?topic=milvus-connecting-service#taskconctmilvus__postreq__1)

In [66]:
# Create index parameters
index_params = milvus_client.prepare_index_params()

# Add first index for text_embedding
index_params.add_index(
    field_name="name_embedding",
    index_type="SPARSE_INVERTED_INDEX",
    metric_type="IP",
    params={"drop_ratio_build": 0.2}
)

# Add second index for context_embedding
index_params.add_index(
    field_name="description_embedding",
    index_type="IVF_SQ8",
    metric_type="L2",
    params={"nlist": 128}
)

## Create Collection and Load Data 

In [67]:
# Create index and load collection
milvus_client.create_collection(
    collection_name=COLLECTION_NAME,
    schema=schema,
    index_params=index_params
)

# Load the collection
milvus_client.load_collection(collection_name=COLLECTION_NAME)

## Generate Embeddings

We are going to generate 2 type of embeddings. One using sentence transformer which is more context aware (sematic), while the second using BM25 - sparse vector embeddings based on TF-IDF focused on keyword search. Click to know more about [Dense](https://github.ibm.com/Aldrin-Dennis1/milvus-enhanced-documentation/blob/main/in-memmory-indexes-and-similarity-metrics.md) and [Sparse](https://github.ibm.com/Aldrin-Dennis1/milvus-enhanced-documentation/blob/main/in-memmory-indexes-and-similarity-metrics-sparse-embeddings.md) vector embeddings or refer [In-Memory Index](https://milvus.io/docs/index.md?tab=floating) 

### 1. Dense embeddings - Sentence Transformer Embeddings - semantic

In [68]:

# Generate embeddings
# print(type(df['description_embedding']))
df['description_embedding'] = df['cleaned_description'].apply(lambda x: model.encode(x).tolist())

# Show the DataFrame with embeddings
df.head(5)


,p_id,name,price,colour,avg_rating,cleaned_description,description_embedding
0,100001,Black Ethnic Motifs Printed Kurta with Palazzo...,5099,Black,4.418398939,Black printed Kurta with Palazzos with dupatta...,"[-0.022137021645903587, 0.08476054668426514, -..."
1,100002,Orange Solid Kurta with Palazzos & Floral Prin...,5899,Orange,4.11933395,Orange solid Kurta with Palazzos with dupattaK...,"[-0.018848655745387077, 0.065238818526268, -0...."
2,100003,Navy Blue Ethnic Motifs Embroidered Thread Wor...,4899,Navy Blue,4.16152968,Navy blue embroidered Kurta with Trousers with...,"[-0.06157408282160759, 0.08424308151006699, 0...."
3,100004,Red Floral Printed Kurta With Trouser & Dupatta,3699,Red,4.088986142,Red printed kurta with trouser and dupattaKurt...,"[-0.05737128108739853, 0.07336778193712234, -0..."
4,100005,Black & Green Printed Straight Kurta,1350,Black,3.978377362,"Black and green printed straight kurta, has a ...","[-0.044882744550704956, 0.088098905980587, 0.0..."


### 2. Sparse Embeddings - BM25 Embeddings - keyword based

In [69]:
corpus = df['name'].tolist()

In [70]:
tokens = []
for i in corpus:
    tokens.append(analyzer(i))
print("tokens:", tokens)

tokens: [['black', 'ethnic', 'motif', 'print', 'kurta', 'palazzo', 'dupatta'], ['orang', 'solid', 'kurta', 'palazzo', 'floral', 'print', 'dupatta'], ['navi', 'blue', 'ethnic', 'motif', 'embroid', 'thread', 'work', 'kurta', 'trouser', 'dupatta'], ['red', 'floral', 'print', 'kurta', 'trouser', 'dupatta'], ['black', 'green', 'print', 'straight', 'kurta'], ['red', 'thread', 'work', 'georgett', 'anarkali', 'kurta'], ['navi', 'blue', 'pure', 'cotton', 'floral', 'print', 'kurta', 'palazzo', 'dupatta'], ['beig', 'floral', 'print', 'regular', 'got', 'ta', 'patti', 'kurta', 'palazzo', 'dupatta'], ['yellow', 'white', 'print', 'kurta', 'palazzo'], ['green', 'pink', 'print', 'pure', 'cotton', 'kurta', 'palazzo', 'dupatta'], ['floral', 'bliss', 'side', 'pocket', 'cotton', 'kurta', 'set'], ['mustard', 'yellow', 'floral', 'yoke', 'embroid', 'straight', 'kurta'], ['teal', 'ethnic', 'motif', 'yoke', 'design', 'kurta', 'trouser', 'dupatta'], ['navi', 'blue', 'floral', 'print', 'regular', 'pure', 'cotton'

In [71]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
bm25_ef = BM25EmbeddingFunction(analyzer)

bm25_ef.fit(corpus)

In [72]:
# Create embeddings for the documents
docs_embeddings = bm25_ef.encode_documents(corpus)

# print("Embeddings:", docs_embeddings)
# Since the output embeddings are in a 2D csr_array format, we convert them to a list for easier manipulation.
#print("Sparse dim:", bm25_ef.dim, list(docs_embeddings)[0].shape)

In [73]:
docs_embeddings.shape

(50, 70)

In [74]:
# Convert into a format the Milvus expects
import numpy as np

def csr_to_dict_list(csr_array):
    result = []
    indptr = csr_array.indptr
    indices = csr_array.indices
    data = csr_array.data

    for i in range(csr_array.shape[0]):
        start, end = indptr[i], indptr[i+1]
        row_indices = indices[start:end]
        row_data = data[start:end]
        row_dict = dict(zip(row_indices, row_data))
        result.append(row_dict)
    
    return result

# Use the function
converted_data = csr_to_dict_list(docs_embeddings)

In [75]:
df['name_embedding'] = converted_data
df = df[['p_id','name','cleaned_description','price','avg_rating','colour','name_embedding','description_embedding']]
df.head(5)

,p_id,name,cleaned_description,price,avg_rating,colour,name_embedding,description_embedding
0,100001,Black Ethnic Motifs Printed Kurta with Palazzo...,Black printed Kurta with Palazzos with dupatta...,5099,4.418398939,Black,"{0: 1.0285399, 1: 1.0285399, 2: 1.0285399, 3: ...","[-0.022137021645903587, 0.08476054668426514, -..."
1,100002,Orange Solid Kurta with Palazzos & Floral Prin...,Orange solid Kurta with Palazzos with dupattaK...,5899,4.11933395,Orange,"{3: 1.0285399, 4: 1.0285399, 5: 1.0285399, 6: ...","[-0.018848655745387077, 0.065238818526268, -0...."
2,100003,Navy Blue Ethnic Motifs Embroidered Thread Wor...,Navy blue embroidered Kurta with Trousers with...,4899,4.16152968,Navy Blue,"{1: 0.8671394, 2: 0.8671394, 4: 0.8671394, 6: ...","[-0.06157408282160759, 0.08424308151006699, 0...."
3,100004,Red Floral Printed Kurta With Trouser & Dupatta,Red printed kurta with trouser and dupattaKurt...,3699,4.088986142,Red,"{3: 1.096575, 4: 1.096575, 6: 1.096575, 9: 1.0...","[-0.05737128108739853, 0.07336778193712234, -0..."
4,100005,Black & Green Printed Straight Kurta,"Black and green printed straight kurta, has a ...",1350,3.978377362,Black,"{0: 1.1742483, 3: 1.1742483, 4: 1.1742483, 17:...","[-0.044882744550704956, 0.088098905980587, 0.0..."


# Ingestion

In [76]:
# Prepare data for insertion as an array of dictionaries
data_to_insert = [
    {
        "p_id": int(row["p_id"]),
        "name": str(row["name"]),
        "cleaned_description": str(row["cleaned_description"]),
        "price": float(row["price"]),
        "avg_rating": float(row["avg_rating"]),
        "colour": row["colour"],
        "name_embedding": row["name_embedding"],
        "description_embedding": row["description_embedding"],
    }
    for _, row in df.iterrows()
]

# Insert data into the Milvus collection
res = milvus_client.insert(
    collection_name=COLLECTION_NAME,
    data=data_to_insert
)

print("Data inserted successfully:", res)

Data inserted successfully: {'insert_count': 50, 'ids': [100001, 100002, 100003, 100004, 100005, 100006, 100007, 100008, 100009, 100010, 100011, 100012, 100013, 100014, 100015, 100016, 100017, 100018, 100019, 100020, 100021, 100022, 100023, 100024, 100025, 100026, 100027, 100028, 100029, 100030, 100031, 100032, 100033, 100034, 100035, 100036, 100037, 100038, 100039, 100040, 100041, 100042, 100043, 100044, 100045, 100046, 100047, 100048, 100049, 100050]}


## Searching with Filter Templating

To use filter templating,define a filter expression with placeholders. During execution, substitute the placeholders with actual values dynamically. This approach avoids embedding large or complex expressions directly into the filter.

more on it [Filter Templating](https://milvus.io/docs/filtering-templating.md#Filter-Templating)

In [77]:
query1 = ["show me cloths that are flowy and with flower designs"]

In [78]:
#semantic search
st_query_embeddings = model.encode(query1)
print("Query text ST Embeddings:", st_query_embeddings.shape)


Query text ST Embeddings: (1, 384)


In [80]:
#use Filter Templating
query_vector = st_query_embeddings
expr = "price < {price} and avg_rating > {avg_rating} and colour in {colour}"
filter_params = {"price": 3000, "avg_rating": 3.5, "colour": ["Black", "Red"]}
res = milvus_client.search(
    collection_name=COLLECTION_NAME,
    data=query_vector,
    limit=3,
    filter=expr, #scalar filter - to keep price lower than 3000 and rating a minimum 3.5 
    output_fields=["name", "price", "cleaned_description"],
    anns_field= "description_embedding",
    filter_params=filter_params
)

for hit in res[0]:
    entity = hit['entity']
    print(f"\nname: {entity['name']}\nPrice: {entity['price']}\ncleaned_description: {entity['cleaned_description']}")


name: Red Floral Print A-Line Kurta
Price: 1699.0
cleaned_description: Red Floral Print A-line kurta, has a round neck, short sleeves, flared hem, has gathers along the waistThe model (height 5'8") is wearing a size S100% viscoseHand-wash

name: Red Bandhani Printed Gotta Patti Kurta
Price: 2999.0
cleaned_description:   Colour: red   Bandhani printed   Round neck   Three-quarter,  regular sleeves   Straight shape with regular style   Gotta patti detail   Calf length with straight hem   Machine weave regular cotton  Cotton  Hand wash The model (height 5'8) is wearing a size S

name: Black & Orange Bandhani Printed Kurta
Price: 1699.0
cleaned_description: Colour: black and orangeBandhani printedMandarin collarButton closureSleeveless, no sleevesA-line shape with regular styleCalf length with flared hemMachine weave regular viscose rayonThe model (height 5'8) is wearing a size SViscose rayon  Machine wash
